## 更新HiRise数据
### HiRise_Update代码原型

In [1]:
import pdr
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
import os

img_url = "https://www.uahirise.org/PDS/DTM/ESP/ORB_045300_045399/ESP_045318_2680_ESP_045346_2680/DTEPC_045318_2680_045346_2680_A01.IMG"

In [2]:
# os.system(r'""C:\Program Files (x86)\Thunder Network\Thunder\Program\ThunderStart.exe"" {url}'.format(url=url))

In [3]:
def check_start(file_name):#判断文件是否开始下载
    tmp=file_name+'.xltd.cfg'
    return os.path.exists(tmp)

def check_end(fiename):#检查文件是否下载成功
    return os.path.exists(fiename)

def download(url,sava_path):#下载文件
    os.system(r'""C:\Program Files (x86)\Thunder Network\Thunder\Program\ThunderStart.exe"" {url}'.format(url=url))#迅雷ThunderStart.exe的路径
    time.sleep(10)
    print("正在下载 {}".format(sava_path.split('/')[-1]))
    if check_start(sava_path):
        while True:
            time.sleep(2)
            if check_end(sava_path):
                return True
    else:
        return False

In [4]:
from tqdm import tqdm


def duan_download(url, file_name):
    r = requests.get(url, stream=True)
    # 获取文件大小
    file_size = int(r.headers['content-length'])
    # 如果文件存在获取文件大小，否在从 0 开始下载，
    first_byte = 0
    if os.path.exists(file_name):
        first_byte = os.path.getsize(file_name)

    # 判断是否已经下载完成
    if first_byte >= file_size:
        return
    # Range 加入请求头
    header = {"Range": f"bytes={first_byte}-{file_size}"}
    # 加了一个 initial 参数
    with tqdm(total=file_size, unit='B', initial=first_byte, unit_scale=True, unit_divisor=1024, ascii=True,
              desc=file_name) as bar:
        # 加 headers 参数
        with requests.get(url, headers=header, stream=True) as r:
            with open(file_name, 'ab') as fp:
                for chunk in r.iter_content(chunk_size=512):
                    if chunk:
                        fp.write(chunk)
                        bar.update(len(chunk))

In [5]:
lbl_url = "https://www.uahirise.org/PDS/INDEX/DTMCUMINDEX.LBL"
tab_url = "https://www.uahirise.org/PDS/INDEX/DTMCUMINDEX.TAB"
lbl_path = "data/DTMCUMINDEX.LBL"
tab_path = "data/DTMCUMINDEX.TAB"
proxy = {'http': 'http://127.0.0.1:9078'}
# 用www.google.com测试代理是否可用
r = requests.get('http://www.google.com', proxies=proxy)
if r.status_code == 200:
    print('代理可用')
else:
    print('代理不可用')

代理可用


In [6]:
# # 获取lbl_url和tab_url的文件大小
# r = requests.get(lbl_url, proxies=proxy)
# lbl_size = int(r.headers['content-length'])
# r = requests.get(tab_url, proxies=proxy)
# tab_size = int(r.headers['content-length'])

In [7]:
# # 如果文件存在且大小相等，不下载
# if os.path.exists(lbl_path) and os.path.exists(tab_path) and os.path.getsize(lbl_path) == lbl_size and os.path.getsize(
#         tab_path) == tab_size:
#     print("文件已存在")
# else:
#     # 下载文件
#     duan_download(lbl_url, lbl_path)
#     duan_download(tab_url, tab_path)

In [8]:
index_tab = pdr.read(lbl_path)
index_tab

pdr.Data(E:\PycharmProjects\mars_sim_web\method\hirise\data\DTMCUMINDEX.LBL)
keys=['LABEL', 'RDR_INDEX_TABLE']
not yet loaded: ('LABEL', 'RDR_INDEX_TABLE')

In [9]:
index_tab = index_tab['RDR_INDEX_TABLE']
index_tab

,VOLUME_ID,FILE_NAME_SPECIFICATION,INSTRUMENT_HOST_ID,INSTRUMENT_ID,PRODUCT_ID,PRODUCT_VERSION_ID,TARGET_NAME,RATIONALE_DESC,LEFT_OBSERVATION_ID,RIGHT_OBSERVATION_ID,...,LINE_PROJECTION_OFFSET,SAMPLE_PROJECTION_OFFSET,CORNER1_LATITUDE,CORNER1_LONGITUDE,CORNER2_LATITUDE,CORNER2_LONGITUDE,CORNER3_LATITUDE,CORNER3_LONGITUDE,CORNER4_LATITUDE,CORNER4_LONGITUDE
0,MROHR_0001,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,MRO,HIRISE,PSP_001336_1560_RED_C_01_ORTHO,1,MARS,Fan/deltaic landform in west Eberswalde Crater...,PSP_001336_1560,NA,...,-1393080.0,3143.5,-23.7216,326.3240,-23.712,326.4130,-23.9584,326.445,-23.9696,326.3570
1,MROHR_0001,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,MRO,HIRISE,DTEEC_001336_1560_001534_1560_U01,1,MARS,Fan/deltaic landform in west Eberswalde Crater...,PSP_001336_1560,PSP_001534_1560,...,-1393080.0,3143.5,-23.7216,326.3240,-23.712,326.4130,-23.9584,326.445,-23.9696,326.3570
2,MROHR_0001,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,MRO,HIRISE,PSP_001336_1560_RED_A_01_ORTHO,1,MARS,Fan/deltaic landform in west Eberswalde Crater...,PSP_001336_1560,NA,...,-5572330.0,12571.5,-23.7216,326.3240,-23.712,326.4130,-23.9584,326.445,-23.9696,326.3570
3,MROHR_0001,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,MRO,HIRISE,PSP_001534_1560_RED_A_01_ORTHO,1,MARS,Fan/deltaic landform in west Eberswalde Crater...,PSP_001534_1560,NA,...,-5572330.0,12571.5,-23.7216,326.3240,-23.712,326.4130,-23.9584,326.445,-23.9696,326.3570
4,MROHR_0001,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,MRO,HIRISE,PSP_001534_1560_RED_C_01_ORTHO,1,MARS,Fan/deltaic landform in west Eberswalde Crater...,PSP_001534_1560,NA,...,-1393080.0,3143.5,-23.7216,326.3240,-23.712,326.4130,-23.9584,326.445,-23.9696,326.3570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9880,MROHR_0001,DTM/ESP/ORB_076500_076599/ESP_076532_0920_ESP_...,MRO,HIRISE,DTEPD_076532_0920_076203_0920_A01,1,MARS,South polar residual cap ...,ESP_076532_0920,ESP_076203_0920,...,-23070.0,57261.0,-87.8860,243.2798,-87.842,243.8538,-87.9248,247.786,-87.9724,247.2666
9881,MROHR_0001,DTM/ESP/ORB_076500_076599/ESP_076532_0920_ESP_...,MRO,HIRISE,ESP_076532_0920_IRB_B_01_ORTHO,1,MARS,South polar residual cap ...,ESP_076532_0920,NA,...,-92278.5,229045.5,-87.8860,243.2798,-87.842,243.8538,-87.9248,247.786,-87.9724,247.2666
9882,MROHR_0001,DTM/ESP/ORB_076500_076599/ESP_076532_0920_ESP_...,MRO,HIRISE,ESP_076532_0920_IRB_D_01_ORTHO,1,MARS,South polar residual cap ...,ESP_076532_0920,NA,...,-23070.0,57261.0,-87.8860,243.2798,-87.842,243.8538,-87.9248,247.786,-87.9724,247.2666
9883,MROHR_0001,DTM/ESP/ORB_076500_076599/ESP_076532_0920_ESP_...,MRO,HIRISE,ESP_076532_0920_RED_B_01_ORTHO,1,MARS,South polar residual cap ...,ESP_076532_0920,NA,...,-92278.5,229045.5,-87.8860,243.2798,-87.842,243.8538,-87.9248,247.786,-87.9724,247.2666


In [10]:
# index_tab['DATA_TYPE'] 去除空格等于DTM的行
index_tab = index_tab[index_tab['DATA_TYPE'].str.strip() == 'DTM']
index_tab

,VOLUME_ID,FILE_NAME_SPECIFICATION,INSTRUMENT_HOST_ID,INSTRUMENT_ID,PRODUCT_ID,PRODUCT_VERSION_ID,TARGET_NAME,RATIONALE_DESC,LEFT_OBSERVATION_ID,RIGHT_OBSERVATION_ID,...,LINE_PROJECTION_OFFSET,SAMPLE_PROJECTION_OFFSET,CORNER1_LATITUDE,CORNER1_LONGITUDE,CORNER2_LATITUDE,CORNER2_LONGITUDE,CORNER3_LATITUDE,CORNER3_LONGITUDE,CORNER4_LATITUDE,CORNER4_LONGITUDE
1,MROHR_0001,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,MRO,HIRISE,DTEEC_001336_1560_001534_1560_U01,1,MARS,Fan/deltaic landform in west Eberswalde Crater...,PSP_001336_1560,PSP_001534_1560,...,-1393080.0,3143.5,-23.7216,326.3240,-23.7120,326.4130,-23.9584,326.4450,-23.9696,326.3570
7,MROHR_0001,DTM/PSP/ORB_001400_001499/PSP_001414_1780_PSP_...,MRO,HIRISE,DTEEC_001414_1780_001612_1780_U01,1,MARS,Possibily Opportunity Rover landing site ...,PSP_001414_1780,PSP_001612_1780,...,-119284.0,396.5,-2.0369,354.4930,-2.0368,354.5120,-2.0647,354.5150,-2.0651,354.4970
12,MROHR_0001,DTM/PSP/ORB_001400_001499/PSP_001432_2015_PSP_...,MRO,HIRISE,DTEEC_001432_2015_001630_2015_U01,1,MARS,Layers in Olympus Mons basal scarp ...,PSP_001432_2015,PSP_001630_2015,...,1267620.0,3429.5,21.5773,222.2980,21.5872,222.3950,21.4577,222.4120,21.4469,222.3150
15,MROHR_0001,DTM/PSP/ORB_001400_001499/PSP_001462_2015_PSP_...,MRO,HIRISE,DTEEC_001462_2015_001792_2015_A01,1,MARS,Cratered cones near Hephaestus Fossae ...,PSP_001462_2015,PSP_001792_2015,...,1260631.5,3308.5,21.4586,123.2702,21.4684,123.3698,21.2591,123.3962,21.2481,123.2981
26,MROHR_0001,DTM/PSP/ORB_001400_001499/PSP_001481_1875_PSP_...,MRO,HIRISE,DTEEC_001481_1875_002167_1880_U01,1,MARS,Dissected wall of Mojave Crater ...,PSP_001481_1875,PSP_002167_1880,...,466260.0,4046.5,7.9462,326.5610,7.9568,326.6520,7.6027,326.6950,7.5914,326.6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9840,MROHR_0001,DTM/ESP/ORB_049900_049999/ESP_049945_0930_ESP_...,MRO,HIRISE,DTEPD_049945_0930_049972_0930_A01,1,MARS,Possible pit at edge of south polar residual c...,ESP_049945_0930,ESP_049972_0930,...,-6940.0,85019.0,-87.1311,262.0184,-87.0898,262.4278,-87.1720,265.2193,-87.2188,264.3801
9849,MROHR_0001,DTM/ESP/ORB_063300_063399/ESP_063381_2485_ESP_...,MRO,HIRISE,DTEPD_063381_2485_063513_2485_A01,1,MARS,Small crater near Phoenix landing site ...,ESP_063381_2485,ESP_063513_2485,...,402200.5,516726.5,68.4526,231.9902,68.4684,232.2249,68.3179,232.2928,68.3033,232.0468
9858,MROHR_0001,DTM/ESP/ORB_063300_063399/ESP_063394_2485_ESP_...,MRO,HIRISE,DTEPD_063394_2485_063526_2485_A01,1,MARS,Pedestal crater ...,ESP_063394_2485,ESP_063526_2485,...,387983.5,533120.5,68.3059,233.8489,68.3193,234.0905,68.1752,234.1542,68.1552,233.9143
9867,MROHR_0001,DTM/ESP/ORB_073100_073199/ESP_073183_0995_ESP_...,MRO,HIRISE,DTEPD_073183_0995_073184_0995_A01,1,MARS,Variety of araneiform terrain ...,ESP_073183_0995,ESP_073184_0995,...,-269901.0,45937.0,-80.4637,188.8443,-80.4572,189.0912,-80.7223,189.5666,-80.7336,189.4047


In [11]:
# 去掉unique()的长度为1的列，即只有一个值的列
index_tab = index_tab.loc[:, index_tab.apply(pd.Series.nunique) != 1]
# 重置index
index_tab = index_tab.reset_index(drop=True)
index_tab

,FILE_NAME_SPECIFICATION,PRODUCT_ID,PRODUCT_VERSION_ID,RATIONALE_DESC,LEFT_OBSERVATION_ID,RIGHT_OBSERVATION_ID,IMAGE_LINES,LINE_SAMPLES,NORTH_AZIMUTH,MINIMUM_LATITUDE,...,LINE_PROJECTION_OFFSET,SAMPLE_PROJECTION_OFFSET,CORNER1_LATITUDE,CORNER1_LONGITUDE,CORNER2_LATITUDE,CORNER2_LONGITUDE,CORNER3_LATITUDE,CORNER3_LONGITUDE,CORNER4_LATITUDE,CORNER4_LONGITUDE
0,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,DTEEC_001336_1560_001534_1560_U01,1,Fan/deltaic landform in west Eberswalde Crater...,PSP_001336_1560,PSP_001534_1560,15166,6789,270.0000,-23.9697,...,-1393080.0,3143.5,-23.7216,326.3240,-23.7120,326.4130,-23.9584,326.4450,-23.9696,326.3570
1,DTM/PSP/ORB_001400_001499/PSP_001414_1780_PSP_...,DTEEC_001414_1780_001612_1780_U01,1,Possibily Opportunity Rover landing site ...,PSP_001414_1780,PSP_001612_1780,1694,1279,270.0000,-2.0652,...,-119284.0,396.5,-2.0369,354.4930,-2.0368,354.5120,-2.0647,354.5150,-2.0651,354.4970
2,DTM/PSP/ORB_001400_001499/PSP_001432_2015_PSP_...,DTEEC_001432_2015_001630_2015_U01,1,Layers in Olympus Mons basal scarp ...,PSP_001432_2015,PSP_001630_2015,8239,6319,270.0000,21.4469,...,1267620.0,3429.5,21.5773,222.2980,21.5872,222.3950,21.4577,222.4120,21.4469,222.3150
3,DTM/PSP/ORB_001400_001499/PSP_001462_2015_PSP_...,DTEEC_001462_2015_001792_2015_A01,1,Cratered cones near Hephaestus Fossae ...,PSP_001462_2015,PSP_001792_2015,12965,6976,270.0000,21.2481,...,1260631.5,3308.5,21.4586,123.2702,21.4684,123.3698,21.2591,123.3962,21.2481,123.2981
4,DTM/PSP/ORB_001400_001499/PSP_001481_1875_PSP_...,DTEEC_001481_1875_002167_1880_U01,1,Dissected wall of Mojave Crater ...,PSP_001481_1875,PSP_002167_1880,21419,7823,270.0000,7.5914,...,466260.0,4046.5,7.9462,326.5610,7.9568,326.6520,7.6027,326.6950,7.5914,326.6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,DTM/ESP/ORB_049900_049999/ESP_049945_0930_ESP_...,DTEPD_049945_0930_049972_0930_A01,1,Possible pit at edge of south polar residual c...,ESP_049945_0930,ESP_049972_0930,4801,3458,173.6200,-87.2189,...,-6940.0,85019.0,-87.1311,262.0184,-87.0898,262.4278,-87.1720,265.2193,-87.2188,264.3801
1012,DTM/ESP/ORB_063300_063399/ESP_063381_2485_ESP_...,DTEPD_063381_2485_063513_2485_A01,1,Small crater near Phoenix landing site ...,ESP_063381_2485,ESP_063513_2485,5840,6130,37.8590,68.3033,...,402200.5,516726.5,68.4526,231.9902,68.4684,232.2249,68.3179,232.2928,68.3033,232.0468
1013,DTM/ESP/ORB_063300_063399/ESP_063394_2485_ESP_...,DTEPD_063394_2485_063526_2485_A01,1,Pedestal crater ...,ESP_063394_2485,ESP_063526_2485,5800,6147,35.9985,68.1552,...,387983.5,533120.5,68.3059,233.8489,68.3193,234.0905,68.1752,234.1542,68.1552,233.9143
1014,DTM/ESP/ORB_073100_073199/ESP_073183_0995_ESP_...,DTEPD_073183_0995_073184_0995_A01,1,Variety of araneiform terrain ...,ESP_073183_0995,ESP_073184_0995,8457,2923,99.2055,-80.7334,...,-269901.0,45937.0,-80.4637,188.8443,-80.4572,189.0912,-80.7223,189.5666,-80.7336,189.4047


In [12]:
# MINIMUM_LATITUDE 和 MAXIMUM_LATITUDE 的mean
mean_lat = index_tab[['MINIMUM_LATITUDE', 'MAXIMUM_LATITUDE']].mean(axis=1)
# MINIMUM_LONGITUDE 和 MAXIMUM_LONGITUDE 的mean
mean_lon = index_tab[['MINIMUM_LONGITUDE', 'MAXIMUM_LONGITUDE']].mean(axis=1)
# CORNER1_LATITUDE 和 CORNER2_LATITUDE 和 CORNER3_LATITUDE 和 CORNER4_LATITUDE 的mean
mean_lat1 = index_tab[['CORNER1_LATITUDE', 'CORNER2_LATITUDE', 'CORNER3_LATITUDE', 'CORNER4_LATITUDE']].mean(axis=1)
# CORNER1_LONGITUDE 和 CORNER2_LONGITUDE 和 CORNER3_LONGITUDE 和 CORNER4_LONGITUDE 的mean
mean_lon1 = index_tab[['CORNER1_LONGITUDE', 'CORNER2_LONGITUDE', 'CORNER3_LONGITUDE', 'CORNER4_LONGITUDE']].mean(
    axis=1)

In [13]:
# 找到abs(mean_lat - mean_lat1) >1 且 abs(mean_lon - mean_lon1) >1 的行,并且去掉这些行
index_tab = index_tab[~((abs(mean_lat - mean_lat1) > 1) & (abs(mean_lon - mean_lon1) > 1))]
index_tab

,FILE_NAME_SPECIFICATION,PRODUCT_ID,PRODUCT_VERSION_ID,RATIONALE_DESC,LEFT_OBSERVATION_ID,RIGHT_OBSERVATION_ID,IMAGE_LINES,LINE_SAMPLES,NORTH_AZIMUTH,MINIMUM_LATITUDE,...,LINE_PROJECTION_OFFSET,SAMPLE_PROJECTION_OFFSET,CORNER1_LATITUDE,CORNER1_LONGITUDE,CORNER2_LATITUDE,CORNER2_LONGITUDE,CORNER3_LATITUDE,CORNER3_LONGITUDE,CORNER4_LATITUDE,CORNER4_LONGITUDE
0,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,DTEEC_001336_1560_001534_1560_U01,1,Fan/deltaic landform in west Eberswalde Crater...,PSP_001336_1560,PSP_001534_1560,15166,6789,270.0000,-23.9697,...,-1393080.0,3143.5,-23.7216,326.3240,-23.7120,326.4130,-23.9584,326.4450,-23.9696,326.3570
1,DTM/PSP/ORB_001400_001499/PSP_001414_1780_PSP_...,DTEEC_001414_1780_001612_1780_U01,1,Possibily Opportunity Rover landing site ...,PSP_001414_1780,PSP_001612_1780,1694,1279,270.0000,-2.0652,...,-119284.0,396.5,-2.0369,354.4930,-2.0368,354.5120,-2.0647,354.5150,-2.0651,354.4970
2,DTM/PSP/ORB_001400_001499/PSP_001432_2015_PSP_...,DTEEC_001432_2015_001630_2015_U01,1,Layers in Olympus Mons basal scarp ...,PSP_001432_2015,PSP_001630_2015,8239,6319,270.0000,21.4469,...,1267620.0,3429.5,21.5773,222.2980,21.5872,222.3950,21.4577,222.4120,21.4469,222.3150
3,DTM/PSP/ORB_001400_001499/PSP_001462_2015_PSP_...,DTEEC_001462_2015_001792_2015_A01,1,Cratered cones near Hephaestus Fossae ...,PSP_001462_2015,PSP_001792_2015,12965,6976,270.0000,21.2481,...,1260631.5,3308.5,21.4586,123.2702,21.4684,123.3698,21.2591,123.3962,21.2481,123.2981
4,DTM/PSP/ORB_001400_001499/PSP_001481_1875_PSP_...,DTEEC_001481_1875_002167_1880_U01,1,Dissected wall of Mojave Crater ...,PSP_001481_1875,PSP_002167_1880,21419,7823,270.0000,7.5914,...,466260.0,4046.5,7.9462,326.5610,7.9568,326.6520,7.6027,326.6950,7.5914,326.6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,DTM/ESP/ORB_049900_049999/ESP_049945_0930_ESP_...,DTEPD_049945_0930_049972_0930_A01,1,Possible pit at edge of south polar residual c...,ESP_049945_0930,ESP_049972_0930,4801,3458,173.6200,-87.2189,...,-6940.0,85019.0,-87.1311,262.0184,-87.0898,262.4278,-87.1720,265.2193,-87.2188,264.3801
1012,DTM/ESP/ORB_063300_063399/ESP_063381_2485_ESP_...,DTEPD_063381_2485_063513_2485_A01,1,Small crater near Phoenix landing site ...,ESP_063381_2485,ESP_063513_2485,5840,6130,37.8590,68.3033,...,402200.5,516726.5,68.4526,231.9902,68.4684,232.2249,68.3179,232.2928,68.3033,232.0468
1013,DTM/ESP/ORB_063300_063399/ESP_063394_2485_ESP_...,DTEPD_063394_2485_063526_2485_A01,1,Pedestal crater ...,ESP_063394_2485,ESP_063526_2485,5800,6147,35.9985,68.1552,...,387983.5,533120.5,68.3059,233.8489,68.3193,234.0905,68.1752,234.1542,68.1552,233.9143
1014,DTM/ESP/ORB_073100_073199/ESP_073183_0995_ESP_...,DTEPD_073183_0995_073184_0995_A01,1,Variety of araneiform terrain ...,ESP_073183_0995,ESP_073184_0995,8457,2923,99.2055,-80.7334,...,-269901.0,45937.0,-80.4637,188.8443,-80.4572,189.0912,-80.7223,189.5666,-80.7336,189.4047


In [14]:
# # 下载所有的DTM
# img_url_prefix = "https://www.uahirise.org/PDS/"
# # http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/DTM/ESP/ORB_045300_045399/ESP_045318_2680_ESP_045346_2680/DTEPC_045318_2680_045346_2680_A01.ca.jpg
# jpg_url_prefix = "http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/"
# img_save_path = "data/DTM/"
# jpg_save_path = "data/JPG/"
# for i in range(index_tab.shape[0]):
#     img_url = img_url_prefix + index_tab.iloc[i]['FILE_NAME_SPECIFICATION']
#     img_file_name = img_save_path + index_tab.iloc[i]['FILE_NAME_SPECIFICATION'].split('/')[-1]
#     duan_download(img_url, img_file_name)
#     # 下载jpg
#     jpg_url = jpg_url_prefix + index_tab.iloc[i]['FILE_NAME_SPECIFICATION'][:-4] + '.ca.jpg'
#     jpg_file_name = jpg_save_path + index_tab.iloc[i]['FILE_NAME_SPECIFICATION'].split('/')[-1][:-4] + '.jpg'
#     duan_download(jpg_url, jpg_file_name)
#     print('第' + str(i) + '个，下载完成：', img_file_name.split('/')[-1], jpg_file_name.split('/')[-1])

In [15]:
# # 下载所有的DTM
# img_url_prefix = "https://www.uahirise.org/PDS/"
# http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/DTM/ESP/ORB_045300_045399/ESP_045318_2680_ESP_045346_2680/DTEPC_045318_2680_045346_2680_A01.ca.jpg
# jpg_url_prefix = "http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/"
# img_save_path = "data/DTM/"
# jpg_save_path = "data/JPG/"
# for i in range(index_tab.shape[0]):
#     img_url = img_url_prefix + index_tab.iloc[i]['FILE_NAME_SPECIFICATION']
#     img_file_name = img_save_path + index_tab.iloc[i]['FILE_NAME_SPECIFICATION'].split('/')[-1]
#     # 等待download()返回True
#     while not download(img_url, img_file_name):
#         print("下载失败，重新下载")
#     # 下载jpg
#     jpg_url = jpg_url_prefix + index_tab.iloc[i]['FILE_NAME_SPECIFICATION'][:-4] + '.ca.jpg'
#     jpg_file_name = jpg_save_path + index_tab.iloc[i]['FILE_NAME_SPECIFICATION'].split('/')[-1][:-4] + '.ca.jpg'
#     duan_download(jpg_url, jpg_file_name)
#     print('第' + str(i) + '个，下载完成：', img_file_name.split('/')[-1], jpg_file_name.split('/')[-1])

In [16]:
import os
import time

def download_with_thunder(urls, save_paths):
    # urls 和 save_paths 组成一个元组的列表
    mission = list(zip(urls, save_paths))
    nums = 40
    
    # 如果任务小于10个，则全部加入mission_to_download，否则只加入10个
    if len(mission) <= nums:
        mission_to_download = mission
        mission = []
    else:
        mission_to_download = mission[:nums]
        mission = mission[nums:]
    
    # 已经启动的任务集合
    started_tasks = set()
    
    while mission_to_download or (mission and any((url, save_path) not in started_tasks for url, save_path in mission)):
        # 检查文件是否已经存在
        to_remove = []
        for url, save_path in mission_to_download:
            if os.path.exists(save_path):
                print(save_path.split('/')[-1] + "已经存在")
                to_remove.append((url, save_path))
            elif (url, save_path) not in started_tasks:
                os.system(r'""C:\Program Files (x86)\Thunder Network\Thunder\Program\ThunderStart.exe"" {url}'.format(url=url))
                started_tasks.add((url, save_path))
                time.sleep(3) # 等待5秒再启动下一个任务
                
        # 从mission_to_download中移除已存在的文件
        for item in to_remove:
            mission_to_download.remove(item)
        
        # 从mission中补充任务到mission_to_download，确保mission_to_download中有10个任务
        while len(mission_to_download) < nums and mission:
            mission_to_download.append(mission.pop(0))
        
        # 每3秒检查一次
        time.sleep(1)

In [17]:
# 下载所有的DTM
img_url_prefix = "https://www.uahirise.org/PDS/"
# http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/DTM/ESP/ORB_045300_045399/ESP_045318_2680_ESP_045346_2680/DTEPC_045318_2680_045346_2680_A01.ca.jpg
jpg_url_prefix = "http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/"
img_save_path = "data/DTM/"
jpg_save_path = "E:/PycharmProjects/mars_sim_web/method/hirise/data/JPG/"
img_urls = img_url_prefix + index_tab['FILE_NAME_SPECIFICATION']
img_urls

0       https://www.uahirise.org/PDS/DTM/PSP/ORB_00130...
1       https://www.uahirise.org/PDS/DTM/PSP/ORB_00140...
2       https://www.uahirise.org/PDS/DTM/PSP/ORB_00140...
3       https://www.uahirise.org/PDS/DTM/PSP/ORB_00140...
4       https://www.uahirise.org/PDS/DTM/PSP/ORB_00140...
                              ...                        
1011    https://www.uahirise.org/PDS/DTM/ESP/ORB_04990...
1012    https://www.uahirise.org/PDS/DTM/ESP/ORB_06330...
1013    https://www.uahirise.org/PDS/DTM/ESP/ORB_06330...
1014    https://www.uahirise.org/PDS/DTM/ESP/ORB_07310...
1015    https://www.uahirise.org/PDS/DTM/ESP/ORB_07650...
Name: FILE_NAME_SPECIFICATION, Length: 1015, dtype: object

In [18]:
img_save_paths = img_save_path + index_tab['FILE_NAME_SPECIFICATION'].str.split('/').str[-1]
img_save_paths

0       data/DTM/DTEEC_001336_1560_001534_1560_U01.IMG
1       data/DTM/DTEEC_001414_1780_001612_1780_U01.IMG
2       data/DTM/DTEEC_001432_2015_001630_2015_U01.IMG
3       data/DTM/DTEEC_001462_2015_001792_2015_A01.IMG
4       data/DTM/DTEEC_001481_1875_002167_1880_U01.IMG
                             ...                      
1011    data/DTM/DTEPD_049945_0930_049972_0930_A01.IMG
1012    data/DTM/DTEPD_063381_2485_063513_2485_A01.IMG
1013    data/DTM/DTEPD_063394_2485_063526_2485_A01.IMG
1014    data/DTM/DTEPD_073183_0995_073184_0995_A01.IMG
1015    data/DTM/DTEPD_076532_0920_076203_0920_A01.IMG
Name: FILE_NAME_SPECIFICATION, Length: 1015, dtype: object

In [19]:
# download_with_thunder(img_urls, img_save_paths)

In [20]:
index_tab

,FILE_NAME_SPECIFICATION,PRODUCT_ID,PRODUCT_VERSION_ID,RATIONALE_DESC,LEFT_OBSERVATION_ID,RIGHT_OBSERVATION_ID,IMAGE_LINES,LINE_SAMPLES,NORTH_AZIMUTH,MINIMUM_LATITUDE,...,LINE_PROJECTION_OFFSET,SAMPLE_PROJECTION_OFFSET,CORNER1_LATITUDE,CORNER1_LONGITUDE,CORNER2_LATITUDE,CORNER2_LONGITUDE,CORNER3_LATITUDE,CORNER3_LONGITUDE,CORNER4_LATITUDE,CORNER4_LONGITUDE
0,DTM/PSP/ORB_001300_001399/PSP_001336_1560_PSP_...,DTEEC_001336_1560_001534_1560_U01,1,Fan/deltaic landform in west Eberswalde Crater...,PSP_001336_1560,PSP_001534_1560,15166,6789,270.0000,-23.9697,...,-1393080.0,3143.5,-23.7216,326.3240,-23.7120,326.4130,-23.9584,326.4450,-23.9696,326.3570
1,DTM/PSP/ORB_001400_001499/PSP_001414_1780_PSP_...,DTEEC_001414_1780_001612_1780_U01,1,Possibily Opportunity Rover landing site ...,PSP_001414_1780,PSP_001612_1780,1694,1279,270.0000,-2.0652,...,-119284.0,396.5,-2.0369,354.4930,-2.0368,354.5120,-2.0647,354.5150,-2.0651,354.4970
2,DTM/PSP/ORB_001400_001499/PSP_001432_2015_PSP_...,DTEEC_001432_2015_001630_2015_U01,1,Layers in Olympus Mons basal scarp ...,PSP_001432_2015,PSP_001630_2015,8239,6319,270.0000,21.4469,...,1267620.0,3429.5,21.5773,222.2980,21.5872,222.3950,21.4577,222.4120,21.4469,222.3150
3,DTM/PSP/ORB_001400_001499/PSP_001462_2015_PSP_...,DTEEC_001462_2015_001792_2015_A01,1,Cratered cones near Hephaestus Fossae ...,PSP_001462_2015,PSP_001792_2015,12965,6976,270.0000,21.2481,...,1260631.5,3308.5,21.4586,123.2702,21.4684,123.3698,21.2591,123.3962,21.2481,123.2981
4,DTM/PSP/ORB_001400_001499/PSP_001481_1875_PSP_...,DTEEC_001481_1875_002167_1880_U01,1,Dissected wall of Mojave Crater ...,PSP_001481_1875,PSP_002167_1880,21419,7823,270.0000,7.5914,...,466260.0,4046.5,7.9462,326.5610,7.9568,326.6520,7.6027,326.6950,7.5914,326.6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,DTM/ESP/ORB_049900_049999/ESP_049945_0930_ESP_...,DTEPD_049945_0930_049972_0930_A01,1,Possible pit at edge of south polar residual c...,ESP_049945_0930,ESP_049972_0930,4801,3458,173.6200,-87.2189,...,-6940.0,85019.0,-87.1311,262.0184,-87.0898,262.4278,-87.1720,265.2193,-87.2188,264.3801
1012,DTM/ESP/ORB_063300_063399/ESP_063381_2485_ESP_...,DTEPD_063381_2485_063513_2485_A01,1,Small crater near Phoenix landing site ...,ESP_063381_2485,ESP_063513_2485,5840,6130,37.8590,68.3033,...,402200.5,516726.5,68.4526,231.9902,68.4684,232.2249,68.3179,232.2928,68.3033,232.0468
1013,DTM/ESP/ORB_063300_063399/ESP_063394_2485_ESP_...,DTEPD_063394_2485_063526_2485_A01,1,Pedestal crater ...,ESP_063394_2485,ESP_063526_2485,5800,6147,35.9985,68.1552,...,387983.5,533120.5,68.3059,233.8489,68.3193,234.0905,68.1752,234.1542,68.1552,233.9143
1014,DTM/ESP/ORB_073100_073199/ESP_073183_0995_ESP_...,DTEPD_073183_0995_073184_0995_A01,1,Variety of araneiform terrain ...,ESP_073183_0995,ESP_073184_0995,8457,2923,99.2055,-80.7334,...,-269901.0,45937.0,-80.4637,188.8443,-80.4572,189.0912,-80.7223,189.5666,-80.7336,189.4047


In [22]:
import method.download_method as dm
file_names = index_tab['FILE_NAME_SPECIFICATION'].str.split('/').str[-1]
img_save_path = "E:/PycharmProjects/mars_sim_web/method/hirise/data/DTM/"
dm.download_with_thunder(img_urls, file_names, img_save_path)

迅雷关闭成功。
所有文件已经存在。


In [23]:
jpg_url = jpg_url_prefix + index_tab['FILE_NAME_SPECIFICATION'].str.replace('.IMG', '.ca.jpg')
jpg_url

0       http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
1       http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
2       http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
3       http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
4       http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
                              ...                        
1011    http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
1012    http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
1013    http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
1014    http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
1015    http://hirise-pds.lpl.arizona.edu/PDS/EXTRAS/D...
Name: FILE_NAME_SPECIFICATION, Length: 1015, dtype: object

In [28]:
file_names = index_tab['FILE_NAME_SPECIFICATION'].str.split('/').str[-1].str.replace('.IMG', '.ca.jpg')
jpg_save_path = "E:/PycharmProjects/mars_sim_web/method/hirise/data/JPG/"
dm.download_with_thunder(jpg_url, file_names, jpg_save_path)

关闭迅雷失败。
所有文件已经存在。


In [41]:
jpg_save_paths = jpg_save_path + index_tab['PRODUCT_ID'] + '.ca.jpg'
jpg_save_paths

0       E:/PycharmProjects/mars_sim_web/method/hirise/...
1       E:/PycharmProjects/mars_sim_web/method/hirise/...
2       E:/PycharmProjects/mars_sim_web/method/hirise/...
3       E:/PycharmProjects/mars_sim_web/method/hirise/...
4       E:/PycharmProjects/mars_sim_web/method/hirise/...
                              ...                        
1011    E:/PycharmProjects/mars_sim_web/method/hirise/...
1012    E:/PycharmProjects/mars_sim_web/method/hirise/...
1013    E:/PycharmProjects/mars_sim_web/method/hirise/...
1014    E:/PycharmProjects/mars_sim_web/method/hirise/...
1015    E:/PycharmProjects/mars_sim_web/method/hirise/...
Name: PRODUCT_ID, Length: 1015, dtype: object

In [22]:
import json
import subprocess
import time
# 文件路径使用原始字符串
thunder_config_path = r"C:\Program Files (x86)\Thunder Network\Thunder\Profiles\config.json"

# 关闭迅雷程序
result = subprocess.run(["taskkill", "/F", "/IM", "Thunder.exe"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
if result.returncode == 0:
    print("Thunder is closed successfully.")
else:
    print("Failed to close Thunder.")

time.sleep(2)

try:
    # 打开json文件，修改下载路径
    with open(thunder_config_path, 'r', encoding='utf-8') as f:
        config = json.load(f)
    config['PathAndCategory']['LastUsedPath'] = jpg_save_path[:-1].replace('/', '\\')

    # 保存修改后的配置
    with open(thunder_config_path, 'w', encoding='utf-8') as f:
        json.dump(config, f)
except FileNotFoundError:
    print("File not found:", thunder_config_path)
except json.JSONDecodeError:
    print("JSON decode error")

# 启动迅雷程序，并等待它完成
try:
    # 启动迅雷程序，不等待其关闭
    subprocess.run([r'C:\Program Files (x86)\Thunder Network\Thunder\Program\Thunder.exe'],check=False)
except FileNotFoundError:
    print("Thunder.exe not found")

# 迅雷程序已经关闭并完成，可以继续执行后续代码
print("Thunder is closed, continue with the rest of the code.")

Failed to close Thunder.
Thunder is closed, continue with the rest of the code.


In [42]:
# 下载所有的jpg
download_with_thunder(jpg_url, jpg_save_paths)

DTEEC_001336_1560_001534_1560_U01.ca.jpg已经存在
DTEEC_001414_1780_001612_1780_U01.ca.jpg已经存在
DTEEC_001432_2015_001630_2015_U01.ca.jpg已经存在
DTEEC_001462_2015_001792_2015_A01.ca.jpg已经存在
DTEEC_001481_1875_002167_1880_U01.ca.jpg已经存在
DTEEC_001488_1750_001752_1750_U02.ca.jpg已经存在
DTEEC_001513_1655_001777_1650_U01.ca.jpg已经存在
DTEEC_001521_2025_001719_2025_U01.ca.jpg已经存在
DTEEC_001546_2015_001955_2015_U01.ca.jpg已经存在
DTEEC_001606_1900_002226_1900_A01.ca.jpg已经存在
DTEEC_001714_1415_001846_1415_U01.ca.jpg已经存在
DTEEC_001871_1965_024419_1965_A01.ca.jpg已经存在
DTEEC_001918_1735_001984_1735_U01.ca.jpg已经存在
DTEEC_001938_2265_002439_2265_U01.ca.jpg已经存在
DTEEC_001981_1825_002258_1825_H01.ca.jpg已经存在
DTEEC_002047_1890_001902_1890_U01.ca.jpg已经存在
DTEEC_002055_2280_001976_2280_A01.ca.jpg已经存在
DTEEC_002063_1735_001641_1735_U01.ca.jpg已经存在
DTEEC_002074_2025_002140_2025_U01.ca.jpg已经存在
DTEEC_002088_1530_002154_1530_U01.ca.jpg已经存在
DTEEC_002118_1510_003608_1510_A01.ca.jpg已经存在
DTEEC_002175_2210_001410_2210_A01.ca.jpg已经存在
DTEEC_0022

KeyboardInterrupt: 

In [44]:
# import os
# 
# # 定义目录路径
# directory_path = "./data/JPG1"
# 
# # 获取目录中的所有文件
# file_list = os.listdir(directory_path)
# file_list

['DTEEC_001336_1560_001534_1560_U01.jpg',
 'DTEEC_001414_1780_001612_1780_U01.jpg',
 'DTEEC_001432_2015_001630_2015_U01.jpg',
 'DTEEC_001462_2015_001792_2015_A01.jpg',
 'DTEEC_001481_1875_002167_1880_U01.jpg',
 'DTEEC_001488_1750_001752_1750_U02.jpg',
 'DTEEC_001513_1655_001777_1650_U01.jpg',
 'DTEEC_001521_2025_001719_2025_U01.jpg',
 'DTEEC_001546_2015_001955_2015_U01.jpg',
 'DTEEC_001606_1900_002226_1900_A01.jpg',
 'DTEEC_001714_1415_001846_1415_U01.jpg',
 'DTEEC_001871_1965_024419_1965_A01.jpg',
 'DTEEC_001918_1735_001984_1735_U01.jpg',
 'DTEEC_001938_2265_002439_2265_U01.jpg',
 'DTEEC_001981_1825_002258_1825_H01.jpg',
 'DTEEC_002047_1890_001902_1890_U01.jpg',
 'DTEEC_002055_2280_001976_2280_A01.jpg',
 'DTEEC_002063_1735_001641_1735_U01.jpg',
 'DTEEC_002074_2025_002140_2025_U01.jpg',
 'DTEEC_002088_1530_002154_1530_U01.jpg',
 'DTEEC_002118_1510_003608_1510_A01.jpg',
 'DTEEC_002175_2210_001410_2210_A01.jpg',
 'DTEEC_002289_1725_002922_1725_A01.jpg',
 'DTEEC_002315_2030_002592_2030_A0

In [45]:
# # 遍历文件列表并重命名
# for index, filename in enumerate(file_list):
#     # 构造新的文件名，原文件名.jpg后缀替换为.ca.jpg
#     new_filename = filename.replace(".jpg", ".ca.jpg")
#     
#     # 构建完整的文件路径
#     old_file_path = os.path.join(directory_path, filename)
#     new_file_path = os.path.join(directory_path, new_filename)
#     
#     # 重命名文件
#     os.rename(old_file_path, new_file_path)
# 
# print("文件重命名完成。")

文件重命名完成。
